In [1]:
#import Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB as gnb
from sklearn.model_selection import train_test_split
import warnings
from sklearn_porter import Porter
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# We are reading our data
df = pd.read_csv("heart.csv")

In [3]:
#process dataset
a = pd.get_dummies(df['cp'], prefix = "cp")
b = pd.get_dummies(df['restecg'], prefix = "restecg")
c = pd.get_dummies(df['slope'], prefix = "slope")
d = pd.get_dummies(df['ca'], prefix = "ca")
e = pd.get_dummies(df['thal'], prefix = "thal")
frames = [df, a, b, c, d, e]
df = pd.concat(frames, axis = 1)
df = df.drop(columns = ['cp', 'restecg', 'slope', 'ca', 'thal'])

In [4]:
#seperate data and target
y = df.target.values
x_data = df.drop(['target'], axis = 1)

In [18]:
# Normalize
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data)).values

In [6]:
#prepare for train test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=6232)

In [7]:
#call model
gnb = gnb()
gnb.fit(x_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [8]:
def prepare_data_form(data):
    data = np.array(data.split(","),dtype=np.float64)
    a = np.zeros(4,dtype=np.float64)
    b = np.zeros(3,dtype=np.float64)
    c = np.zeros(3,dtype=np.float64)
    d = np.zeros(5,dtype=np.float64)
    e = np.zeros(4,dtype=np.float64)
    for i in range(4):
        if data[2]==i:
            a[i]=1
    for i in range(3):
        if data[6]==i:
            b[i]=1
    for i in range(3):
        if data[10]==i:
            c[i]=1
    for i in range(5):
        if data[11]==i:
            d[i]=1
    for i in range(4):
        if data[12]==i:
            e[i]=1
    data = np.delete(data,12)
    data = np.delete(data,11)
    data = np.delete(data,10)
    data = np.delete(data,6)
    data = np.delete(data,2)
    data = np.insert(data,len(data),a)
    data = np.insert(data,len(data),b)
    data = np.insert(data,len(data),c)
    data = np.insert(data,len(data),d)
    data = np.insert(data,len(data),e)
    data = data.reshape(1,-1)
    new_data = (data - np.min(data)) / (np.max(data) - np.min(data))
    return new_data

In [9]:
accuracies = {}
acc = gnb.score(x_train,y_train)*100
accuracies['GaussianNB'] = acc
print("Train Accuracy {:.2f}%".format(acc))

Train Accuracy 83.06%


In [10]:
acc2 = gnb.score(x_test,y_test)*100
accuracies['GaussianNB'] = acc2
print("Test Accuracy {:.2f}%".format(acc2))

Test Accuracy 96.72%


result = []
result2 = []
for i in range(10000):
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=i)
    x_train = x_train.T
    y_train = y_train.T
    x_test = x_test.T
    y_test = y_test.T
    accuracies = {}
    ml = gnb()
    ml.fit(x_train.T,y_train.T)
    acc = ml.score(x_train.T,y_train.T)*100
    acc2 = ml.score(x_test.T,y_test.T)*100
    result.append(acc+acc2)
    
print(max(result),result.index(max(result)))

In [14]:
#form = "0.70833333,1,0.48113208,0.24429224,1,0.60305344,0,0.37096774,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0" #1
form = "0.16666667,1,0.33962264,0.28310502,0,0.88549618,0,0.56451613,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0" #1<<<
#form = "0.583333,0,0.339623,0.251142,0,0.786260,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0" #0
#form = "0.583333,1,0.339623,0.011416,0,0.335878,1,0.193548,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1" #0<<<
form = np.array(form.split(","),dtype=np.float64)
form = form.reshape(1,-1)

#form_norm = prepare_data_form(form)
result = "{:.2f}%".format(gnb.predict_proba(form)[0,1]*100)
print(result)
print(gnb.predict_proba(form))

98.03%
[[0.01974236 0.98025764]]


In [12]:
porter = Porter(gnb, language='java')
output = porter.export(embed_data=True)
print(output)

class GaussianNB {

    private double[] priors;
    private double[][] sigmas;
    private double[][] thetas;

    public GaussianNB(double[] priors, double[][] sigmas, double[][] thetas) {
        this.priors = priors;
        this.sigmas = sigmas;
        this.thetas = thetas;
    }

    public int predict(double[] features) {
        double[] likelihoods = new double[this.sigmas.length];
    
        for (int i = 0, il = this.sigmas.length; i < il; i++) {
            double sum = 0.;
            for (int j = 0, jl = this.sigmas[0].length; j < jl; j++) {
                sum += Math.log(2. * Math.PI * this.sigmas[i][j]);
            }
            double nij = -0.5 * sum;
            sum = 0.;
            for (int j = 0, jl = this.sigmas[0].length; j < jl; j++) {
                sum += Math.pow(features[j] - this.thetas[i][j], 2.) / this.sigmas[i][j];
            }
            nij -= 0.5 * sum;
            likelihoods[i] = Math.log(this.priors[i]) + nij;
        }
    
        int cla